In [1]:
#importing the Libraies
import numpy as np
import pandas as pd

In [2]:
# Reading the Dataset
dataset = pd.read_csv('/content/Social_Network_Ads.csv')

In [3]:
dataset.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [4]:
dataset.tail()

,User ID,Gender,Age,EstimatedSalary,Purchased
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0
399,15594041,Female,49,36000,1


In [5]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [6]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [7]:
X=dataset[['Age', 'EstimatedSalary','Gender_Male']]
y=dataset['Purchased']

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [9]:
from sklearn.linear_model import LogisticRegression

In [10]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [11]:
from sklearn.model_selection import GridSearchCV, cross_val_predict

# Define parameter grid (smaller for speed)
param_grid = {
    'penalty': ['l1', 'l2'],               # Regularization type
    'C': [0.01, 0.1, 1, 10],              # Inverse regularization strength
    'solver': ['liblinear', 'saga'],      # Solvers supporting l1 & l2
    'max_iter': [200]                     # Iterations
}


grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')

# fitting the model for grid search
grid.fit(X,y)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10], 'max_iter': [200],
                         'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'saga']},
             scoring='f1_weighted', verbose=3)

In [12]:
# Cross-validated predictions (instead of X_test)
y_pred = cross_val_predict(grid.best_estimator_, X, y, cv=5)

In [13]:
# Evaluation
from sklearn.metrics import confusion_matrix, classification_report, f1_score
cm = confusion_matrix(y, y_pred)
clf_report = classification_report(y, y_pred)
f1_macro = f1_score(y, y_pred, average='weighted')

In [14]:
print("Best Parameters:", grid.best_params_)
print("The f1_macro:", f1_macro)
print("The confusion Matrix:\n", cm)
print("The report:\n", clf_report)

Best Parameters: {'C': 0.1, 'max_iter': 200, 'penalty': 'l1', 'solver': 'liblinear'}
The f1_macro: 0.8228163992869875
The confusion Matrix:
 [[229  28]
 [ 42 101]]
The report:
               precision    recall  f1-score   support

           0       0.85      0.89      0.87       257
           1       0.78      0.71      0.74       143

    accuracy                           0.82       400
   macro avg       0.81      0.80      0.81       400
weighted avg       0.82      0.82      0.82       400



In [15]:
# Save Best Model
import pickle
filename = "LogisticRegression_CV_best_model.sav"
pickle.dump(grid.best_estimator_, open(filename, 'wb'))

In [16]:
# CV Results table
results_df = pd.DataFrame(grid.cv_results_)
results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_max_iter,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.011856,0.012428,0.008878,0.006196,0.01,200,l1,liblinear,"{'C': 0.01, 'max_iter': 200, 'penalty': 'l1', ...",0.512121,0.512121,0.496374,0.677033,0.513443,0.542218,0.067700,15
1,0.006199,0.002482,0.010359,0.002841,0.01,200,l1,saga,"{'C': 0.01, 'max_iter': 200, 'penalty': 'l1', ...",0.512121,0.512121,0.496374,0.496374,0.496374,0.502673,0.007715,16
2,0.002545,0.000354,0.009202,0.004282,0.01,200,l2,liblinear,"{'C': 0.01, 'max_iter': 200, 'penalty': 'l2', ...",0.629132,0.962338,0.923637,0.815688,0.731251,0.812409,0.122506,2
3,0.006229,0.002469,0.009108,0.003820,0.01,200,l2,saga,"{'C': 0.01, 'max_iter': 200, 'penalty': 'l2', ...",0.533333,0.866895,0.741250,0.791273,0.673432,0.721237,0.113239,14
4,0.003802,0.003158,0.005407,0.002928,0.10,200,l1,liblinear,"{'C': 0.1, 'max_iter': 200, 'penalty': 'l1', '...",0.650400,0.962338,0.923637,0.779316,0.751667,0.813472,0.114784,1
